# Test of Curie-Meter-Rem Rule


In [1]:
import openmc
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [2]:
air = openmc.Material(name="air");
air.add_element('O',0.2);
air.add_element('N',0.8);
air.set_density('g/cm3',1.3e-3);

materials = openmc.Materials([air]);

In [3]:
s1 = openmc.Sphere(r=10.0);
s2 = openmc.Sphere(r=20.0);
s3 = openmc.Sphere(r=50.0);
s4 = openmc.Sphere(r=80.0);
s5 = openmc.Sphere(r=100.0);
s6 = openmc.Sphere(r=200.0, boundary_type='vacuum');

c1 = openmc.Cell();
c1.region = -s1;
c1.fill = air;

c2 = openmc.Cell();
c2.region = +s1 & -s2;
c2.fill = air;

c3 = openmc.Cell();
c3.region = +s2 & -s3;
c3.fill = air;

c4 = openmc.Cell();
c4.region = +s3 & -s4;
c4.fill = air;

c5 = openmc.Cell();
c5.region = +s4 & -s5;
c5.fill = air;

c6 = openmc.Cell();
c6.region = +s5 & -s6;
c6.fill = air;

root_univ = openmc.Universe();
root_univ.add_cells([c1,c2,c3,c4,c5,c6]);

geometry = openmc.Geometry();
geometry.root_universe = root_univ;

materials.export_to_xml();
geometry.export_to_xml();

In [4]:
flux = openmc.Tally(name = 'flux');
energy,dose = openmc.data.dose_coefficients('photon','ISO');
dose_filter = openmc.EnergyFunctionFilter(energy,dose);
surface_filter = openmc.SurfaceFilter(s5);

flux.filters = [dose_filter,surface_filter];
flux.scores = ['current'];
tallies = openmc.Tallies([flux]);
tallies.export_to_xml();


In [5]:
print(energy)

[1.000e+04 1.500e+04 2.000e+04 3.000e+04 4.000e+04 5.000e+04 6.000e+04
 7.000e+04 8.000e+04 1.000e+05 1.500e+05 2.000e+05 3.000e+05 4.000e+05
 5.000e+05 5.110e+05 6.000e+05 6.620e+05 8.000e+05 1.000e+06 1.117e+06
 1.330e+06 1.500e+06 2.000e+06 3.000e+06 4.000e+06 5.000e+06 6.000e+06
 6.129e+06 8.000e+06 1.000e+07 1.500e+07 2.000e+07 3.000e+07 4.000e+07
 5.000e+07 6.000e+07 8.000e+07 1.000e+08 1.500e+08 2.000e+08 3.000e+08
 4.000e+08 5.000e+08 6.000e+08 8.000e+08 1.000e+09 1.500e+09 2.000e+09
 3.000e+09 4.000e+09 5.000e+09 6.000e+09 8.000e+09 1.000e+10]


In [6]:
print(dose)

[2.88e-02 5.60e-02 8.13e-02 1.27e-01 1.58e-01 1.80e-01 1.98e-01 2.18e-01
 2.38e-01 2.86e-01 4.29e-01 5.89e-01 9.32e-01 1.28e+00 1.63e+00 1.66e+00
 1.97e+00 2.17e+00 2.62e+00 3.25e+00 3.60e+00 4.21e+00 4.67e+00 5.91e+00
 8.08e+00 1.00e+01 1.18e+01 1.35e+01 1.37e+01 1.66e+01 1.97e+01 2.68e+01
 3.38e+01 4.61e+01 5.69e+01 6.61e+01 7.41e+01 8.71e+01 9.75e+01 1.16e+02
 1.29e+02 1.47e+02 1.59e+02 1.67e+02 1.74e+02 1.85e+02 1.93e+02 2.08e+02
 2.18e+02 2.32e+02 2.42e+02 2.51e+02 2.58e+02 2.68e+02 2.76e+02]


In [7]:
point = openmc.stats.Point();

# create a discrete PMF - 50% chance each for 1.17 Mev and 1.33 MeV photons
energy_probs = openmc.stats.Discrete([1.17e6, 1.33e6],[0.5,0.5]);

source = openmc.Source(space=point,energy=energy_probs,particle='photon');

settings = openmc.Settings();
settings.run_mode = 'fixed source';
settings.batches = 100;
settings.particles = 30000;
settings.generations_per_batch = 100;
settings.source = source;

settings.export_to_xml();

In [8]:
openmc.run();

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [9]:
sp = openmc.StatePoint('statepoint.100.h5');

dose_t = sp.get_tally(name='flux');
dose_df = dose_t.get_pandas_dataframe();
dose_df.head()

,energyfunction,surface,nuclide,score,mean,std. dev.
0,b8e3496a2a8dd2,5,total,current,3.967549,0.000162


In [10]:
current = dose_df['mean'];

source_strength = 3.7e10 * 2.0; # photons per second
conv_pSv_to_rem = 1e-10;
surf_area = 4*np.pi*(100**2); # cm^2 for the given surface
time = 3600; # seconds in an hour

current = current*conv_pSv_to_rem*source_strength*time/surf_area

In [11]:
print('the dose is approximately %6.4g rem/hr' % current)

the dose is approximately 0.8411 rem/hr
